<a href="https://colab.research.google.com/github/urmilapol/urmilapolprojects/blob/master/Tourism1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import urllib.request
import os
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords


In [5]:

df = pd.read_csv('/content/sample_data/hotel review/train.csv', encoding='utf-8')
df.head(3)


,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy


In [7]:
import nltk


In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:

# init Objects
tokenizer=RegexpTokenizer(r'\w+')
en_stopwords=set(stopwords.words('english'))
ps=PorterStemmer()
def getStemmedReview(review):
    review=review.lower()
    review=review.replace("<br /><br />"," ")
    #Tokenize
    tokens=tokenizer.tokenize(review)
    new_tokens=[token for token in tokens if token not in  en_stopwords]
    stemmed_tokens=[ps.stem(token) for token in new_tokens]
    clean_review=' '.join(stemmed_tokens)
    return clean_review

In [10]:
df['Is_Response'] = df['Is_Response'].map({'not happy':0, 'happy':1})

In [11]:
df.head(10)

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,0
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,0
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,0
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,1
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,0
5,id10331,We had - rooms. One was very nice and clearly ...,InternetExplorer,Desktop,1
6,id10332,My husband and I have stayed in this hotel a f...,Firefox,Tablet,0
7,id10333,My wife & I stayed in this glorious city a whi...,Google Chrome,Mobile,1
8,id10334,My boyfriend and I stayed at the Fairmont on a...,Internet Explorer,Desktop,1
9,id10335,"Wonderful staff, great location, but it was de...",Chrome,Tablet,0


In [12]:
df.drop(['User_ID','Browser_Used','Device_Used'],axis=1,inplace=True)



In [13]:
df.head(10)

,Description,Is_Response
0,The room was kind of clean but had a VERY stro...,0
1,I stayed at the Crown Plaza April -- - April -...,0
2,I booked this hotel through Hotwire at the low...,0
3,Stayed here with husband and sons on the way t...,1
4,My girlfriends and I stayed here to celebrate ...,0
5,We had - rooms. One was very nice and clearly ...,1
6,My husband and I have stayed in this hotel a f...,0
7,My wife & I stayed in this glorious city a whi...,1
8,My boyfriend and I stayed at the Fairmont on a...,1
9,"Wonderful staff, great location, but it was de...",0


In [19]:
df['Description'].apply(getStemmedReview)
X_train = df.loc[:35000, 'Description'].values
y_train = df.loc[:35000, 'Is_Response'].values
X_test = df.loc[35000:, 'Description'].values
y_test = df.loc[35000:, 'Is_Response'].values



In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, encoding='utf-8',decode_error='ignore')
vectorizer.fit(X_train)
X_train=vectorizer.transform(X_train)
X_test=vectorizer.transform(X_test)

In [21]:
import numpy as np

# Here 1 denotes a positive sentiment
model.predict(X_test[0])

# 78% probability that the given text is negative
model.predict_proba(X_test[0])

import joblib

joblib.dump(en_stopwords,'stopwords.pkl') 
joblib.dump(model,'model.pkl')
joblib.dump(vectorizer,'vectorizer.pkl')




['vectorizer.pkl']

In [24]:
!pip install flask-ngrok
!pip install -U WTForms

     |████████████████████████████████| 174kB 4.0MB/s 


In [27]:
!pip install portpicker

In [47]:
!pip install WSGIServer

In [45]:
!pip install --user gevent

In [50]:
pip install wsgiserver

In [57]:
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import wsgiserver
import pickle
import sqlite3
import os
import numpy as np
import joblib
loaded_model=joblib.load("/content/model.pkl")
loaded_stop=joblib.load("/content/stopwords.pkl")
loaded_vec=joblib.load("/content/vectorizer.pkl")
app = Flask(__name__)


def classify(document):
    label = {0: 'negative', 1: 'positive'}
    X = loaded_vec.transform([document])
    y = loaded_model.predict(X)[0]
    proba = np.max(loaded_model.predict_proba(X))
    return label[y], proba

class ReviewForm(Form):
    Hotelreview = TextAreaField(' ',[validators.DataRequired(),validators.length(min=15)])
    
@app.route('/')


def index():
    form = ReviewForm(request.form)
    return render_template('/content/sample_data/hotel review/reviewform.html', form=form)

@app.route('/content/sample_data/hotel review/reviewform.html', methods=['POST'])
def results():
    form = ReviewForm(request.form)
    if request.method == 'POST' and form.validate():
        review = request.form['Hotelreview']
        y, proba = classify(review)
        return render_template('results.html',content=review,prediction=y,probability=round(proba*100, 2))
    return render_template('reviewform.html', form=form)

if __name__ == '__main__':
  